# ML Assignment 2: Optimization

## Overview

In developing this script, I referenced
 Hayes, G. (2019). mlrose: Machine Learning, Randomized Optimization and SEarch package for Python. https://github.com/gkhayes/mlrose 
 and also https://github.com/hiive/mlrose

mlrose is a Python package for applying some of the most common randomized optimization and search algorithms to a range of different optimization problems, over both discrete- and continuous-valued parameter spaces. This notebook contains the examples used in the mlrose tutorial.

### Import Libraries

In [1]:
import time
import threading

import mlrose_hiive as mlrose
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

### Problem 1:  Fill the Knapsack!

In [2]:
class Runner:
    def __init__(self,n, problem_name, algs, alg_names):
        self.n = n
        self.name = problem_name
        fig1, ax1 = plt.subplots()
#         ax1.title = "Score Per Iteration"
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
#         ax2.title = "Evals Per Iteration"
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.algs = algs
        self.names = alg_names
        self.times = []
        self.best_fitnesses = []
        
    def run(self):
        for algorithm, name in zip(self.algs,self.names):
            print(f'starting {name}....')
            tic = time.perf_counter()
            best_state, best_fitness, fitness_curve = algorithm()
            toc = time.perf_counter()
#             print(f"{name} completed in {toc - tic:0.4f} seconds")
#             print(f'{name} found best fitness/value: ', best_fitness, '\n')
            
            self.times.append(toc-tic)
            self.best_fitnesses.append(best_fitness)
            self.plot_curve(fitness_curve, f'{name}')
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    def print_stats(self):
        for name, time, best_fitness in zip(self.names, self.times, self.best_fitnesses):
            print(f'{name} had \n\tTime: {time:0.4f} seconds\n\tFitness Score: {best_fitness}')
    def plot_curve(self, fitness_curve, name):
        self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
        self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [3]:
# Initialize fitness function object using pre-defined class
'''
Fitness function for Knapsack optimization problem. Given a set of n
items, where item i has known weight :math:`w_{i}` and known value
:math:`v_{i}`; and maximum knapsack capacity, :math:`W`, the Knapsack
fitness function evaluates the fitness of a state vector
:math:`x = [x_{0}, x_{1}, \ldots, x_{n-1}]` as:
'''
# https://en.wikipedia.org/wiki/Knapsack_problem
# We're trying to get to the highest value possible, without going over our weight limit

ns = [50,100, 200, 400, 800, 1000,1500] # items in our knapsack
weights = []
values = []
fitnesses = []
init_states = []
for n in ns:
    weight = (np.random.randint(1,50,size=(n)))
    value = (np.random.randint(1,50,size=(n)))
    init_state = np.random.randint(0,1,size=(n))
    init_states.append(init_state)
    max_weight_pct = 0.6 # so we can only hold 60% of our total weight for items we're trying to fit
    fitnesses.append(mlrose.Knapsack(weight, value, max_weight_pct))
    weights.append(weight)
    values.append(value)

# basically have a seed state so we can reproduce
random_state = 5
fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
for weight, value, fitness, n, init_state in zip(weights, values, fitnesses, ns, init_states):
    print('=====================')
    print(f'starting n = {n}...')
    algs = []
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    schedule = mlrose.GeomDecay(.01,0.0001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem, schedule=schedule, curve=True, random_state = random_state))
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
 

    thing = Runner(n, 'knapsack', algs, alg_names)
    thing.run()
    thing.print_stats()
    
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    print('\n')
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/knapsack_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/knapsack_times_problem_size.png', bbox_inches='tight')
fig1.clf()

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0015 seconds
	Fitness Score: 734.0
simulated_annealing had 
	Time: 0.0019 seconds
	Fitness Score: 801.0
genetic_algorithm had 
	Time: 0.5517 seconds
	Fitness Score: 1148.0
mimic had 
	Time: 0.3290 seconds
	Fitness Score: 1131.0


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0009 seconds
	Fitness Score: 1477.0
simulated_annealing had 
	Time: 0.0010 seconds
	Fitness Score: 1476.0
genetic_algorithm had 
	Time: 0.8313 seconds
	Fitness Score: 2235.0
mimic had 
	Time: 0.8857 seconds
	Fitness Score: 2205.0


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....
random_hill_climbing had 
	Time: 0.0032 seconds
	Fitness S

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Part 2: Six Peaks 

In [ ]:
# Initialize fitness function object using pre-defined class

ns = [50,100,200, 300, 500, 800,1000,1500] # items in our knapsack

fitnesses = []
for n in ns:
    fitnesses.append(mlrose.SixPeaks(t_pct=0.20))


# basically have a seed state so we can reproduce


# basically have a seed state so we can reproduce
random_state = 5

fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
    
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))
    schedule = mlrose.GeomDecay(.01,0.001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem,schedule=schedule, curve=True, random_state = random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, random_state=random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    
    thing = Runner(n, 'six_peaks', algs, alg_names)
    thing.run()
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    print('\n')
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/six_peaks_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/six_peaks_times_problem_size.png', bbox_inches='tight')
fig1.clf()

starting n = 50...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 100...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 200...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 300...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 500...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 800...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 1000...
starting random_hill_climbing....
starting simulated_annealing....
starting genetic_algorithm....
starting mimic....


starting n = 1500...
starti

### Example 3: Queens

In [ ]:
ns = [50,100,200,400,500,800] # items in our knapsack

fitnesses = []
init_coords = []
for n in ns:
    # Define alternative N-Queens fitness function for maximization problem
    # This code was ripped off from https://mlrose.readthedocs.io/en/stable/source/tutorial1.html#solving-optimization-problems-with-mlrose
    def queens_max(state):
       # Initialize counter
        fitness_cnt = 0
          # For all pairs of queens
        for i in range(len(state) - 1):
            for j in range(i + 1, len(state)):
                # Check for horizontal, diagonal-up and diagonal-down attacks
                if (state[j] != state[i]) and (state[j] != state[i] + (j - i)) and (state[j] != state[i] - (j - i)):
                   # If no attacks, then increment counter
                    fitness_cnt += 1
        return fitness_cnt

    # Initialize custom fitness function object
    fitness_cust = mlrose.CustomFitness(queens_max)
    fitnesses.append(fitness_cust)


# basically have a seed state so we can reproduce
random_state = 5
fitness_values= dict()
times = dict()
alg_names = ['random_hill_climbing','simulated_annealing','genetic_algorithm','mimic']
for name in alg_names:
    fitness_values[name] = []
    times[name] = []
    
for fitness, n in zip(fitnesses, ns):
    print('=====================')
    print(f'starting n = {n}...')
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs = []
    
    algs.append(lambda: mlrose.random_hill_climb(problem, curve=True, random_state = random_state))

    
    schedule = mlrose.GeomDecay(.01,0.001)
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.simulated_annealing(problem, curve=True, random_state = random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    algs.append(lambda: mlrose.genetic_alg(problem, curve=True, max_iters=100, random_state=random_state))
    problem = mlrose.DiscreteOpt(length = n, fitness_fn = fitness, maximize=True, max_val=2)
    problem.set_mimic_fast_mode(True)
    algs.append(lambda: mlrose.mimic(problem,
                                     curve=True, 
                                     random_state=random_state))
    
    thing = Runner(n, 'queens', algs, alg_names)
    thing.run()
    thing.print_stats()
    for index, name in enumerate(alg_names):
        fitness_values[name].append(thing.best_fitnesses[index])
        times[name].append(thing.times[index])
    print('\n')
    
fig1, ax1 = plt.subplots()
ax1.set_ylabel("Fitness score")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, fitness_values[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/queens_problem_size.png', bbox_inches='tight')
fig1.clf()

fig1, ax1 = plt.subplots()
ax1.set_ylabel("Seconds to Convergence")
ax1.set_xlabel("Problem Size")

for name in alg_names:
    ax1.plot(ns, times[name], label=name)

fig1.legend(loc=4)
fig1.savefig(f'charts/queens_times_problem_size.png', bbox_inches='tight')
fig1.clf()

### Example 6: Fitting a Neural Network to the Heart Dataset

In [ ]:

df = pd.read_csv("data/heart.csv")
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]
 # https://www.kaggle.com/ronitf/heart-disease-uci


In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 3)

In [ ]:
# Normalize feature data
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# One hot encode target values
one_hot = OneHotEncoder()

y_train_hot = one_hot.fit_transform(np.array(y_train).reshape(-1, 1)).todense()
y_test_hot = one_hot.transform(np.array(y_test).reshape(-1, 1)).todense()

In [ ]:
class NNRunner:
    def __init__(self, alg_names):
        fig1, ax1 = plt.subplots()
        ax1.set_ylabel("Fitness score")
        ax1.set_xlabel("Iteration")
        self.fig1 = fig1
        self.ax1 = ax1
        
        fig2, ax2 = plt.subplots()
        ax2.set_ylabel("Fitness Evals")
        ax2.set_xlabel("Iterations")
        self.fig2 = fig2
        self.ax2 = ax2
        
        self.names = alg_names
        self.name = "NN"
        self.times = []
        self.scores = []
        
        
    # Copied from https://scikit-learn.org/stable/auto_examples/applications/plot_model_complexity_influence.html
    def benchmark_influence(self, conf):
        """
        Benchmark influence of `changing_param` on both MSE and latency.
        """
        prediction_times = []
        prediction_powers = []
        complexities = []
        for param_value in conf['changing_param_values']:
            conf['tuned_params'][conf['changing_param']] = param_value
            estimator = conf['estimator'](**conf['tuned_params'])

            print("Benchmarking %s" % estimator)
            estimator.fit(conf['data']['X_train'], conf['data']['y_train'])
            conf['postfit_hook'](estimator)
            complexity = conf['complexity_computer'](estimator)
            complexities.append(complexity)
            start_time = time.time()
            for _ in range(conf['n_samples']):
                y_pred = estimator.predict(conf['data']['X_test'])
            elapsed_time = (time.time() - start_time) / float(conf['n_samples'])
            prediction_times.append(elapsed_time)
            pred_score = conf['prediction_performance_computer'](
                conf['data']['y_test'], y_pred)
            prediction_powers.append(pred_score)
            print("Complexity: %d | %s: %.4f | Pred. Time: %fs\n" % (
                complexity, conf['prediction_performance_label'], pred_score,
                elapsed_time))
        return prediction_powers, prediction_times, complexities
    
    def _count_nonzero_coefficients(estimator):
        a = estimator.coef_.toarray()
        return np.count_nonzero(a)


    configurations = [
        {'estimator': SGDClassifier,
         'tuned_params': {'penalty': 'elasticnet', 'alpha': 0.001, 'loss':
                          'modified_huber', 'fit_intercept': True, 'tol': 1e-3},
         'changing_param': 'l1_ratio',
         'changing_param_values': [0.25, 0.5, 0.75, 0.9],
         'complexity_label': 'non_zero coefficients',
         'complexity_computer': _count_nonzero_coefficients,
         'prediction_performance_computer': hamming_loss,
         'prediction_performance_label': 'Hamming Loss (Misclassification Ratio)',
         'postfit_hook': lambda x: x.sparsify(),
         'data': classification_data,
         'n_samples': 30},
        {'estimator': NuSVR,
         'tuned_params': {'C': 1e3, 'gamma': 2 ** -15},
         'changing_param': 'nu',
         'changing_param_values': [0.1, 0.25, 0.5, 0.75, 0.9],
         'complexity_label': 'n_support_vectors',
         'complexity_computer': lambda x: len(x.support_vectors_),
         'data': regression_data,
         'postfit_hook': lambda x: x,
         'prediction_performance_computer': mean_squared_error,
         'prediction_performance_label': 'MSE',
         'n_samples': 30},
        {'estimator': GradientBoostingRegressor,
         'tuned_params': {'loss': 'squared_error'},
         'changing_param': 'n_estimators',
         'changing_param_values': [10, 50, 100, 200, 500],
         'complexity_label': 'n_trees',
         'complexity_computer': lambda x: x.n_estimators,
         'data': regression_data,
         'postfit_hook': lambda x: x,
         'prediction_performance_computer': mean_squared_error,
         'prediction_performance_label': 'MSE',
         'n_samples': 30},
    ]
    def run(self):
        for name in self.names:
            print(f'starting {name}....')
            tic = time.perf_counter()
            print('===============================================')
            print(f'running NN weight optimization for {name}')
            # Initialize neural network object and fit object - attempt 1


            schedule = mlrose.GeomDecay(.01,0.0001) 
            nn = mlrose.NeuralNetwork(hidden_nodes = [2], activation ='relu', 
                                             algorithm =name, 
                                             max_iters = 100, bias = False, is_classifier = True, 
                                             learning_rate = 0.0001, early_stopping = True,
                                             schedule=schedule,
                                             curve=True,
                                             clip_max = 5, max_attempts = 1000, random_state = 5)
            _, axes = plt.subplots(1, 1, figsize=(20, 5))
            axes.set_title("NN")
            axes.set_xlabel("Training examples")
            axes.set_ylabel("Score")
            train_sizes, train_scores, test_scores, fit_times, _ = \
                    learning_curve(nn, X_train_scaled, y_train_hot,return_times=True)

            train_scores_mean = np.mean(train_scores, axis=1)
            train_scores_std = np.std(train_scores, axis=1)
            test_scores_mean = np.mean(test_scores, axis=1)
            test_scores_std = np.std(test_scores, axis=1)
            fit_times_mean = np.mean(fit_times, axis=1)
            fit_times_std = np.std(fit_times, axis=1)

            # Plot learning curve
            axes.grid()
            axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                                 train_scores_mean + train_scores_std, alpha=0.1,
                                 color="r")
            axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                                 test_scores_mean + test_scores_std, alpha=0.1,
                                 color="g")
            axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                         label="Training score")
            axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                         label="Cross-validation score")
            axes.legend(loc="best")

            plt.savefig(f'charts/NN_{self.name}_{name}.png', bbox_inches='tight') 
            plt.clf()
        
        self.fig1.legend(loc=4)
        self.fig2.legend(loc=4)
        self.fig1.savefig(f'charts/{self.name}_{n}_items.png', bbox_inches='tight')
        self.fig2.savefig(f'charts/{self.name}_{n}_items_evals.png', bbox_inches='tight')
        self.fig1.clf()
        self.fig2.clf()
    
    def print_stats(self):
        for name, time, best_fitness in zip(self.names, self.times, self.scores):
            print(f'{name} had \n\tTime: {time:0.4f} seconds\n\t Score: {score}')
    def plot_curve(self, fitness_curve, name):
        if len(fitness_curve.shape) == 2:
            self.ax1.plot(range(0,len(fitness_curve)), fitness_curve[:,0], label=f'{name}')
            self.ax2.plot(range(0,len(fitness_curve)), fitness_curve[:,1], label=f'{name}')
    


In [ ]:
algorithms = ['random_hill_climb', 'simulated_annealing', 'genetic_alg','gradient_descent']

runner_for_nn = NNRunner(algorithms)

runner_for_nn.run()

runner_for_nn.print_stats()
    

    